In [ ]:
import requests
import os
import json
from dotenv import load_dotenv

# Load .env file
load_dotenv()

# Load the API key
API_KEY = os.getenv("MORALIS_KEY")
if not API_KEY:
    raise ValueError("Server API Key not loaded. Check your .env file.")

# Transaction hash (with or without 0x prefix)
TRANSACTION_HASH = "0xfeda0e8f0d6e54112c28d319c0d303c065d1125c9197bd653682f5fcb0a6c81e"

# Build the URL
url = f"https://deep-index.moralis.io/api/v2.2/transaction/{TRANSACTION_HASH}/verbose?chain=eth"

headers = {
    "Accept": "application/json",
    "X-API-Key": API_KEY
}

try:
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        tx_data = response.json()
        print(json.dumps(tx_data, indent=2))
        
        # Example: Extract useful information
        print("\n--- Transaction Summary ---")
        print(f"From: {tx_data['from_address_label']} ({tx_data['from_address']})")
        print(f"To: {tx_data['to_address_label']} ({tx_data['to_address']})")
        print(f"Block: {tx_data['block_number']}")
        print(f"Status: {'Success' if tx_data['receipt_status'] == '1' else 'Failed'}")
        print(f"Gas Used: {tx_data['receipt_gas_used']}")
        print(f"Transaction Fee: {tx_data['transaction_fee']} ETH")
        
        # Extract decoded events
        if tx_data.get('logs'):
            print(f"\n--- Events ({len(tx_data['logs'])} total) ---")
            for log in tx_data['logs']:
                if log.get('decoded_event'):
                    event = log['decoded_event']
                    print(f"{event['label']}: {event['signature']}")
    else:
        print(f"Error: {response.status_code}")
        print(response.text)
except Exception as e:
    print(f"Request failed: {e}")

{
  "hash": "0xfeda0e8f0d6e54112c28d319c0d303c065d1125c9197bd653682f5fcb0a6c81e",
  "nonce": "327",
  "transaction_index": "176",
  "from_address_entity": null,
  "from_address_entity_logo": null,
  "from_address": "0xcb1c1fde09f811b294172696404e88e658659905",
  "from_address_label": "reubenrjs.eth",
  "to_address_entity": "1inch",
  "to_address_entity_logo": "https://entities-logos.s3.us-east-1.amazonaws.com/1inch.png",
  "to_address": "0x111111125421ca6dc452d289314280a0f8842a65",
  "to_address_label": "1inch: AggregationRouterV6",
  "value": "0",
  "gas": "217574",
  "gas_price": "848440939",
  "input": "0x07ed23790000000000000000000000005141b82f5ffda4c6fe1e372978f1c5427640a190000000000000000000000000a0b86991c6218b36c1d19d4a2e9eb0ce3606eb48000000000000000000000000eeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee0000000000000000000000005141b82f5ffda4c6fe1e372978f1c5427640a190000000000000000000000000cb1c1fde09f811b294172696404e88e65865990500000000000000000000000000000000000000000000000000000000

In [12]:
import requests
import json
from dotenv import load_dotenv
import os
from datetime import datetime

load_dotenv()

API_KEY = os.getenv("MORALIS_KEY")
if not API_KEY:
    raise ValueError("MORALIS_KEY not found in .env file")

# Wallet address to query
WALLET_ADDRESS = "0xcB1C1FdE09f811B294172696404e88E658659905"

# Build URL for wallet transactions
url = f"https://deep-index.moralis.io/api/v2.2/{WALLET_ADDRESS}/verbose?chain=eth&order=DESC&limit=25"

headers = {
    "Accept": "application/json",
    "X-API-Key": API_KEY
}

try:
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        data = response.json()
        
        # Print pagination info
        print(f"Page: {data.get('page')}/{data.get('page_size')} transactions per page")
        print(f"Total transactions fetched: {len(data.get('result', []))}\n")
        
        # Extract and display transaction summaries
        if data.get('result'):
            print("=" * 120)
            for idx, tx in enumerate(data['result'], 1):
                print(f"\n{idx}. Transaction Hash: {tx['hash']}")
                print(f"   From: {tx['from_address_label'] or tx['from_address']}")
                print(f"   To: {tx['to_address_label'] or tx['to_address']}")
                print(f"   Value: {float(tx['value']) / 1e18:.4f} ETH")
                print(f"   Block: {tx['block_number']}")
                print(f"   Time: {tx['block_timestamp']}")
                print(f"   Status: {'✓ Success' if tx['receipt_status'] == '1' else '✗ Failed'}")
                print(f"   Gas Used: {tx['receipt_gas_used']}")
                print(f"   Fee: {float(tx['transaction_fee']):.6f} ETH")
                
                # Show decoded function call if available
                if tx.get('decoded_call'):
                    call = tx['decoded_call']
                    print(f"   Function: {call['label']} ({call['signature']})")
                
                # Show events
                if tx.get('logs'):
                    decoded_events = [log for log in tx['logs'] if log.get('decoded_event')]
                    if decoded_events:
                        print(f"   Events: {len(decoded_events)}")
                        for event in decoded_events[:2]:  # Show first 2 events
                            print(f"     - {event['decoded_event']['label']}")
            
            print("\n" + "=" * 120)
            
            # Handle pagination
            if data.get('cursor'):
                print(f"\nMore transactions available. Use cursor: {data['cursor'][:50]}...")
                print("To fetch next page, add: &cursor={cursor_value}")
        else:
            print("No transactions found")
    else:
        print(f"Error: {response.status_code}")
        print(response.text)
        
except Exception as e:
    print(f"Request failed: {e}")

Page: 1/25 transactions per page
Total transactions fetched: 25


1. Transaction Hash: 0xf53ef12d0b1604acee0a30b27e025980a3370e51cf087581ae8adc3800f3d0ae
   From: reubenrjs.eth
   To: 0x4fef9d741011476750a243ac70b9789a63dd47df
   Value: 0.0010 ETH
   Block: 23583751
   Time: 2025-10-15T14:41:23.000Z
   Status: ✓ Success
   Gas Used: 71112
   Fee: 0.000176 ETH
   Function: deposit (deposit(address,address))

2. Transaction Hash: 0x2672ff7ec16848ea46b2269ea5a138a67215fb62550e41a39e41c1098b2a871b
   From: reubenrjs.eth
   To: OpenSea: Seaport
   Value: 0.0019 ETH
   Block: 23371336
   Time: 2025-09-15T22:03:35.000Z
   Status: ✓ Success
   Gas Used: 194278
   Fee: 0.000291 ETH
   Function: fulfillBasicOrder_efficient_6GL6yc (fulfillBasicOrder_efficient_6GL6yc(tuple))
   Events: 2
     - OrderFulfilled
     - Transfer

3. Transaction Hash: 0xe630b9487f89a8256265c0565701a75e11e52d8a942a26f4714741659a90b654
   From: reubenrjs.eth
   To: Socket: Socket Gateway
   Value: 0.0160 ETH
   Block: 22

In [3]:
import requests
import json
from dotenv import load_dotenv
import os
from datetime import datetime

load_dotenv()

API_KEY = os.getenv("MORALIS_KEY")
if not API_KEY:
    raise ValueError("MORALIS_KEY not found in .env file")

# Wallet address to query
WALLET_ADDRESS = "0xcB1C1FdE09f811B294172696404e88E658659905"

# Build URL for wallet history
url = f"https://deep-index.moralis.io/api/v2.2/wallets/{WALLET_ADDRESS}/history?chain=eth&order=DESC&limit=25"

headers = {
    "Accept": "application/json",
    "X-API-Key": API_KEY
}

try:
    response = requests.get(url, headers=headers)
    
    if response.status_code == 200:
        data = response.json()
        
        # Print pagination info
        print(f"Page: {data.get('page')}/{data.get('page_size')} transactions per page")
        print(f"Total transactions fetched: {len(data.get('result', []))}\n")
        
        # Extract and display transaction summaries
        if data.get('result'):
            print("=" * 120)
            for idx, tx in enumerate(data['result'], 1):
                print(f"\n{idx}. Transaction Hash: {tx['hash']}")
                print(f"   From: {tx['from_address_label'] or tx['from_address']}")
                print(f"   To: {tx['to_address_label'] or tx['to_address']}")
                print(f"   Value: {float(tx['value']) / 1e18:.6f} ETH")
                print(f"   Block: {tx['block_number']}")
                print(f"   Time: {tx['block_timestamp']}")
                print(f"   Status: {'✓ Success' if tx['receipt_status'] == '1' else '✗ Failed'}")
                print(f"   Gas Used: {tx['receipt_gas_used']}")
                print(f"   Fee: {float(tx['transaction_fee']):.6f} ETH")
                print(f"   Category: {tx.get('category', 'N/A')}")
                print(f"   Summary: {tx.get('summary', 'N/A')}")
                
                # Show method label if available
                if tx.get('method_label'):
                    print(f"   Method: {tx['method_label']}")
                
                # Show NFT transfers
                if tx.get('nft_transfers'):
                    print(f"   NFT Transfers: {len(tx['nft_transfers'])}")
                    for nft in tx['nft_transfers'][:2]:  # Show first 2
                        direction = nft.get('direction', 'N/A')
                        token_id = nft.get('token_id', 'N/A')
                        print(f"     - {direction.upper()}: Token #{token_id}")
                
                # Show ERC20 transfers
                if tx.get('erc20_transfers'):
                    print(f"   Token Transfers: {len(tx['erc20_transfers'])}")
                    for token in tx['erc20_transfers'][:2]:  # Show first 2
                        symbol = token.get('token_symbol', 'Unknown')
                        value = token.get('value_formatted', '0')
                        direction = token.get('direction', 'N/A')
                        print(f"     - {direction.upper()}: {value} {symbol}")
                
                # Show native transfers
                if tx.get('native_transfers'):
                    print(f"   Native Transfers: {len(tx['native_transfers'])}")
                    for transfer in tx['native_transfers'][:2]:  # Show first 2
                        value = transfer.get('value_formatted', '0')
                        direction = transfer.get('direction', 'N/A')
                        print(f"     - {direction.upper()}: {value} ETH")
            
            print("\n" + "=" * 120)
            
            # Transaction category summary
            categories = {}
            for tx in data['result']:
                cat = tx.get('category', 'unknown')
                categories[cat] = categories.get(cat, 0) + 1
            
            print("\nTransaction Categories:")
            for cat, count in sorted(categories.items(), key=lambda x: x[1], reverse=True):
                print(f"  {cat}: {count}")
            
            # Handle pagination
            if data.get('cursor'):
                print(f"\n{'='*120}")
                print(f"More transactions available. Use cursor: {data['cursor'][:50]}...")
                print("To fetch next page, add: &cursor={cursor_value}")
        else:
            print("No transactions found")
    else:
        print(f"Error: {response.status_code}")
        print(response.text)
        
except Exception as e:
    print(f"Request failed: {e}")


# Function to fetch multiple pages
def fetch_all_pages(wallet_address, max_pages=5):
    """Fetch multiple pages of transaction history"""
    all_transactions = []
    cursor = None
    page_num = 0
    
    while page_num < max_pages:
        page_url = f"https://deep-index.moralis.io/api/v2.2/wallets/{wallet_address}/history?chain=eth&order=DESC&limit=25"
        if cursor:
            page_url += f"&cursor={cursor}"
        
        try:
            response = requests.get(page_url, headers=headers)
            if response.status_code == 200:
                data = response.json()
                all_transactions.extend(data.get('result', []))
                cursor = data.get('cursor')
                page_num += 1
                
                print(f"Fetched page {page_num}, total transactions: {len(all_transactions)}")
                
                if not cursor:
                    break
            else:
                print(f"Error on page {page_num + 1}: {response.status_code}")
                break
        except Exception as e:
            print(f"Error fetching page {page_num + 1}: {e}")
            break
    
    return all_transactions

Page: 0/25 transactions per page
Total transactions fetched: 25


1. Transaction Hash: 0xf53ef12d0b1604acee0a30b27e025980a3370e51cf087581ae8adc3800f3d0ae
   From: reubenrjs.eth
   To: 0x4fef9d741011476750a243ac70b9789a63dd47df
   Value: 0.001000 ETH
   Block: 23583751
   Time: 2025-10-15T14:41:23.000Z
   Status: ✓ Success
   Gas Used: 71112
   Fee: 0.000176 ETH
   Category: send
   Summary: Sent 0.001 ETH to 0x4f...47df
   Method: deposit
   Native Transfers: 1
     - SEND: 0.001 ETH

2. Transaction Hash: 0x2672ff7ec16848ea46b2269ea5a138a67215fb62550e41a39e41c1098b2a871b
   From: reubenrjs.eth
   To: OpenSea: Seaport
   Value: 0.001855 ETH
   Block: 23371336
   Time: 2025-09-15T22:03:35.000Z
   Status: ✓ Success
   Gas Used: 194278
   Fee: 0.000291 ETH
   Category: nft purchase
   Summary: Purchased 1 NFT
   Method: fulfillBasicOrder_efficient_6GL6yc
   NFT Transfers: 1
     - RECEIVE: Token #6993
   Native Transfers: 2
     - SEND: 0.001855 ETH
     - SEND: 0.001855 ETH

3. Transactio

In [ ]:
# Blockchain Forensics APP with Deep Analysis

from fastapi import FastAPI, HTTPException
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
from typing import List, Dict, Optional, Any
from datetime import datetime, timedelta
from collections import defaultdict
import requests
import os
from dotenv import load_dotenv

# Load environment variables from the .env file
load_dotenv()

# Initialize the FastAPI application with metadata
app = FastAPI(
    title="Blockchain Forensics & Compliance APP",
    version="1.0.0",
    description="Advanced transaction monitoring with deep event analysis and pattern detection"
)

# Configure CORS (Cross-Origin Resource Sharing) middleware
# This allows the frontend (likely running on a different port/domain) to communicate with this backend
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # Allow all origins (for development)
    allow_credentials=True, # Allow sending cookies/credentials
    allow_methods=["*"],  # Allow all HTTP methods (GET, POST, etc.)
    allow_headers=["*"],  # Allow all headers
)

MORALIS_API_KEY = os.getenv("MORALIS_KEY")
MORALIS_BASE_URL = "https://deep-index.moralis.io/api/v2.2"

if not MORALIS_API_KEY or MORALIS_API_KEY == "MORALIS_KEY":
    raise ValueError("❌ MORALIS_KEY not found in .env file")

# Define a dictionary of sanctioned addresses (e.g., Tornado Cash, known hackers)
SANCTIONS_LIST = {
    "0x8576acc5c05d6ce88f4e49bf65bde93d537e45d1": "OFAC Sanctioned - Tornado Cash",
    "0x722122df12d4e14e13ac3b6895a86e84145b6967": "OFAC Sanctioned - Tornado Cash",
    "0xdd4c48c0b24039969fc16d1cdf626eab821d3384": "Sanctioned - Exchange Hack",
    "0x267be1c1d684f78cb4f6a176c4911b741e4ffdc0": "OFAC Sanctioned - Lazarus Group",
    "0x098b716b8aaf21512996dc57eb0615e2383e2f96": "Sanctioned - Mixer Service"
}

# Known Mixer Addresses (Static list for guaranteed detection of common mixers)
MIXER_ADDRESSES = {
    "0x8576acc5c05d6ce88f4e49bf65bde93d537e45d1": "Tornado Cash Router",
    "0x722122df12d4e14e13ac3b6895a86e84145b6967": "Tornado Cash",
    "0x098b716b8aaf21512996dc57eb0615e2383e2f96": "Privacy Mixer",
    "0xd90e2f925da726b50c4ed8d0fb90ad053324f31b": "Tornado Cash Pool",
    "0x12d66f87a04a9e220743712ce6d9bb1b5616b8fc": "Tornado Cash Pool"
}

# Known Exchange Addresses (Static list)
KNOWN_EXCHANGES = {
    "0x3f5ce5fbfe3e9af3971dd833d26ba9b5c936f0be": "Binance Hot Wallet",
    "0x28c6c06298d514db089934071355e5743bf21d60": "Binance 14",
    "0x21a31ee1afc51d94c2efccaa2092ad1028285549": "Binance 15",
    "0xdfd5293d8e347dfe59e90efd55b2956a1343963d": "Binance 16",
    "0x5041ed759dd4afc3a72b8192c143f72f4724081a": "Kraken Exchange",
    "0x267be1c1d684f78cb4f6a176c4911b741e4ffdc0": "Bitfinex Hot Wallet"
}

# High-Risk Method Patterns (Identify suspicious smart contract interactions)
HIGH_RISK_METHODS = {
    "delegatecall", "selfdestruct", "create2", "suicide",
    "callcode", "destroy", "destroyAndSend"
}

# Mixer/Privacy Protocol Identifiers (Keywords to check against labels)
MIXER_KEYWORDS = ["tornado", "mixer", "tumbler", "privacy", "blender", "anonymizer"]

# Pydantic Models for Data Validation and Schema Documentation
class TokenFlow(BaseModel):
    token_address: str         
    token_symbol: Optional[str] 
    from_address: str         
    to_address: str            
    amount: str                
    amount_formatted: str     
    hop_number: int           

class EventAnalysis(BaseModel):
    event_type: str           
    count: int                
    risk_flags: List[str]      
    details: List[Dict[str, Any]] 

class TransactionDetails(BaseModel):
    from_address: str
    from_label: Optional[str]
    from_entity: Optional[str]
    to_address: Optional[str]
    to_label: Optional[str]
    to_entity: Optional[str]
    value: str                 
    block_number: int
    block_timestamp: str       
    gas_used: str
    gas_price: str
    transaction_fee: str
    nonce: int                 # Transaction count for sender
    decoded_call: Optional[Dict] # Details of the function called
    token_flows: List[TokenFlow] 
    total_hops: int            # Total number of transfers

class AnalysisResult(BaseModel):
    tx_hash: str
    risk_score: int            # Calculated risk (0-100)
    risk_level: str            # LOW, MEDIUM, HIGH, CRITICAL
    risk_factors: List[str]    # Explanations for risk score
    flags: List[str]           # Specific warning flags
    details: TransactionDetails
    event_analysis: List[EventAnalysis]
    sanctions_check: bool      # True if involves sanctioned entity
    mixer_interaction: bool    # True if involves mixer
    exchange_interaction: bool # True if involves exchange
    entity_labels: List[str]   # Labels found for involved addresses
    complexity_score: int      # Score based on tx complexity
    timing_flags: List[str]    # Alerts regarding timing (e.g., late night)

class TimePattern(BaseModel):
    tx_per_hour: float
    burst_detected: bool       # True if rapid succession of txs
    suspicious_timing: bool    # True if unusual hours
    time_details: str

class AddressTransaction(BaseModel):
    hash: str
    block_timestamp: str
    from_address: str
    to_address: str
    value: str
    risk_score: int
    flags: List[str]
    entity_interaction: Optional[str]

class AddressAnalysis(BaseModel):
    address: str
    address_label: Optional[str] # Known label for the address
    total_transactions: int
    risk_score: int
    risk_level: str
    risk_factors: List[str]
    flags: List[str]
    entity_labels: List[str]
    recent_transactions: List[AddressTransaction]
    high_risk_counterparties: List[str] # List of risky addresses interacted with
    sanctions_check: bool
    mixer_interaction: bool
    time_patterns: TimePattern
    behavior_summary: Dict[str, Any] # Aggregate behavioral stats

class DashboardTransaction(BaseModel):
    id: str
    hash: str
    block_timestamp: str
    from_address: str
    to_address: str
    value: str
    date: str
    category: str
    risk_score: int
    flags: List[str]
    entity_interaction: Optional[str]

class DashboardStats(BaseModel):
    total_in: str
    total_out: str
    swaps: int
    nfts: int
    incoming_count: int
    outgoing_count: int

class GraphNode(BaseModel):
    id: str
    label: str
    sublabel: str
    type: str
    size: float
    color: str
    glow: Optional[bool] = False

class GraphLink(BaseModel):
    source: str
    target: str
    value: str
    label: str
    type: str

class GraphData(BaseModel):
    nodes: List[GraphNode]
    links: List[GraphLink]

class DashboardResponse(BaseModel):
    success: bool
    address: str
    page: int
    page_size: int
    cursor: Optional[str]
    total_fetched: int
    stats: DashboardStats
    transactions: Dict[str, List[DashboardTransaction]]
    graph_data: GraphData
    raw_count: int

# Helper Functions
def moralis_request(endpoint: str, params: Dict = None) -> Dict:
    """Make request to Moralis API with error handling"""
    headers = {
        "X-API-Key": MORALIS_API_KEY,
        "accept": "application/json"
    }
    url = f"{MORALIS_BASE_URL}{endpoint}"
    
    try:
        response = requests.get(url, headers=headers, params=params, timeout=15)
        if response.status_code == 404:
            raise HTTPException(status_code=404, detail="Resource not found on the specified chain")
        response.raise_for_status()
        return response.json()
    except requests.exceptions.HTTPError as e:
        if e.response.status_code == 404:
             raise HTTPException(status_code=404, detail="Resource not found on the specified chain")
        raise HTTPException(status_code=e.response.status_code, detail=f"Moralis API error: {str(e)}")
    except requests.exceptions.RequestException as e:
        raise HTTPException(status_code=500, detail=f"Moralis API error: {str(e)}")

def check_sanctions(address: str) -> tuple[bool, Optional[str]]:
    """Check if address is on OFAC sanctions list"""
    # Normalize address to lowercase for comparison
    addr_lower = address.lower()
    # Check against local static list
    if addr_lower in SANCTIONS_LIST:
        return True, SANCTIONS_LIST[addr_lower]
    return False, None

def extract_moralis_labels(tx_data: Dict) -> tuple[List[str], bool, bool]:
    """Extract entity labels from Moralis response + static lists (layered approach)"""
    labels = []
    is_mixer = False
    is_exchange = False
    
    # Get addresses from transaction data
    from_addr = tx_data.get("from_address", "").lower()
    to_addr = tx_data.get("to_address", "").lower()
    
    # Check static lists first
    if from_addr in MIXER_ADDRESSES:
        labels.append(f"🔄 From: {MIXER_ADDRESSES[from_addr]} (static list)")
        is_mixer = True
    if to_addr in MIXER_ADDRESSES:
        labels.append(f"🔄 To: {MIXER_ADDRESSES[to_addr]} (static list)")
        is_mixer = True
    
    # Check if addresses are known exchanges (from static list)
    if from_addr in KNOWN_EXCHANGES:
        labels.append(f"🏦 From: {KNOWN_EXCHANGES[from_addr]} (static list)")
        is_exchange = True
    if to_addr in KNOWN_EXCHANGES:
        labels.append(f"🏦 To: {KNOWN_EXCHANGES[to_addr]} (static list)")
        is_exchange = True
    
    # Check sender label
    if tx_data.get("from_address_label"):
        from_label = tx_data["from_address_label"]
        labels.append(f"📤 From: {from_label}")
        if any(kw in from_label.lower() for kw in MIXER_KEYWORDS):
            is_mixer = True
        if "exchange" in from_label.lower():
            is_exchange = True
    
    # Check sender entity name
    if tx_data.get("from_address_entity"):
        from_entity = tx_data["from_address_entity"]
        labels.append(f"🏢 From Entity: {from_entity}")
    
    # Check receiver label
    if tx_data.get("to_address_label"):
        to_label = tx_data["to_address_label"]
        labels.append(f"📥 To: {to_label}")
        if any(kw in to_label.lower() for kw in MIXER_KEYWORDS):
            is_mixer = True
        if "exchange" in to_label.lower() or "binance" in to_label.lower() or "kraken" in to_label.lower():
            is_exchange = True
    
    # Check receiver entity name
    if tx_data.get("to_address_entity"):
        to_entity = tx_data["to_address_entity"]
        labels.append(f"🏢 To Entity: {to_entity}")
        if any(kw in to_entity.lower() for kw in MIXER_KEYWORDS):
            is_mixer = True
    
    return labels, is_mixer, is_exchange

def analyze_events(logs: List[Dict]) -> tuple[List[EventAnalysis], List[str]]:
    """Deep analysis of all transaction events (logs)"""
    event_counts = defaultdict(list)
    flags = []
    
    # Iterate through all event logs
    for log in logs:
        decoded = log.get("decoded_event")
        # Skip if event wasn't decoded (unknown signature)
        if not decoded:
            continue
        
        # Get event name (e.g., Transfer, Approval)
        event_type = decoded.get("label", "Unknown")
        # Extract parameters into a dictionary
        params = {p["name"]: p["value"] for p in decoded.get("params", [])}
        
        # Group events by type
        event_counts[event_type].append({
            "address": log.get("address"),
            "params": params,
            "log_index": log.get("log_index")
        })
    
    # Analyze each event type group
    analyses = []
    
    for event_type, occurrences in event_counts.items():
        event_flags = []
        
        # Check for dangerous Approvals (infinite allowance)
        if event_type == "Approval":
            # Check for very large amounts (near uint256 max)
            unlimited = sum(1 for e in occurrences 
                          if int(e["params"].get("amount", 0)) > 1e50)
            if unlimited > 0:
                event_flags.append(f"{unlimited} unlimited approval(s)")
                flags.append(f"⚠️ {unlimited} unlimited token approval(s) detected")
        
        # Check for complex Transfer patterns
        elif event_type == "Transfer":
            # Count how many distinct tokens were moved
            unique_tokens = len(set(e["address"] for e in occurrences))
            if unique_tokens > 5:
                event_flags.append(f"Multiple tokens: {unique_tokens}")
                flags.append(f"🔄 Complex flow: {unique_tokens} different tokens")
        
        # Check for Swaps (DEX interactions)
        elif event_type in ["Swap", "Swapped"]:
            # Check if there are many swaps in one tx (routing?)
            if len(occurrences) > 3:
                event_flags.append(f"Multi-hop swap: {len(occurrences)} swaps")
                flags.append(f"🔄 Complex DEX routing: {len(occurrences)} swap hops")
        
        # Check for large Withdrawals
        elif event_type == "Withdrawal":
            # Identify large outbound native token moves
            large_withdrawals = sum(1 for e in occurrences 
                                   if int(e["params"].get("wad", 0)) > 1e20) # > 100 ETH
            if large_withdrawals > 0:
                event_flags.append(f"{large_withdrawals} large withdrawal(s)")
        
        # Append analysis for this event type
        analyses.append(EventAnalysis(
            event_type=event_type,
            count=len(occurrences),
            risk_flags=event_flags,
            details=occurrences[:3]  # Return details for First 3 occurrences only
        ))
    
    return analyses, flags

def build_token_flow_graph(logs: List[Dict]) -> tuple[List[TokenFlow], int, List[str]]:
    """Build a graph of token flows and detect cyclic patterns"""
    flows = []
    flags = []
    hop_number = 0
    seen_addresses = set()
    
    # Iterate through logs to find Transfer events
    for log in logs:
        decoded = log.get("decoded_event")
        # Only interested in standard ERC20 Transfers
        if not decoded or decoded.get("label") != "Transfer":
            continue
        
        params = {p["name"]: p["value"] for p in decoded.get("params", [])}
        # Ensure 'from' and 'to' params exist
        if not all(k in params for k in ["from", "to"]):
            continue
        
        hop_number += 1
        from_addr = params["from"]
        to_addr = params["to"]
        # Handle 'value' vs 'amount' naming for different standards
        amount = params.get("value", params.get("amount", 0))
        
        # Track addresses for circular flow detection (Wash Trading Pattern)
        seen_addresses.add(from_addr.lower())
        if to_addr.lower() in seen_addresses:
            flags.append("⚠️ Circular token flow detected (potential wash trading)")
        seen_addresses.add(to_addr.lower())
        
        # Format amount to human-readable (assuming 18 decimals default)
        try:
            amount_int = int(amount)
            decimals = 18  # Default, could be improved with token metadata fetch
            amount_formatted = f"{amount_int / (10 ** decimals):.6f}"
        except:
            amount_formatted = str(amount)
        
        # Add to flow list
        flows.append(TokenFlow(
            token_address=log.get("address", ""),
            token_symbol=log.get("token_symbol"),
            from_address=from_addr,
            to_address=to_addr,
            amount=str(amount),
            amount_formatted=amount_formatted,
            hop_number=hop_number
        ))
    
    return flows, hop_number, flags

def analyze_timing(timestamp_str: str) -> List[str]:
    """Analyze transaction timing for suspicious patterns (e.g., late night hours)"""
    flags = []
    
    try:
        # standardizing timestamp format
        dt = datetime.fromisoformat(timestamp_str.replace("Z", "+00:00"))
        hour = dt.hour
        
        # Check for Late night activity (2 AM - 5 AM UTC) - often correlates with hack timing
        if 2 <= hour <= 5:
            flags.append(f"🌙 Late-night activity ({hour:02d}:00 UTC)")
        
        # Check for Weekend activity (banks closed, less monitoring)
        if dt.weekday() >= 5:
            flags.append("📅 Weekend transaction")
    except:
        pass
    
    return flags

def calculate_complexity_score(token_flows: List[TokenFlow], event_count: int) -> int:
    """Calculate a numeric score (0-100) representing transaction complexity"""
    score = 0
    
    # Points for number of token hops
    score += min(len(token_flows) * 5, 30)
    
    # Points for number of events
    score += min(event_count * 2, 20)
    
    # Points for number of unique addresses involved
    unique_addrs = len(set(
        [f.from_address for f in token_flows] + 
        [f.to_address for f in token_flows]
    ))
    score += min(unique_addrs * 3, 25)
    
    # Cap score at 100
    return min(score, 100)

def calculate_advanced_risk_score(
    sanctions_hit: bool,
    mixer_hit: bool,
    flags: List[str],
    entity_labels: List[str],
    complexity: int,
    timing_flags: List[str]
) -> tuple[int, List[str]]:
    """Determine final risk score based on weighted factors"""
    score = 0
    factors = []
    
    # CRITICAL RISK FACTORS
    if sanctions_hit:
        score += 70
        factors.append("🚨 CRITICAL: Sanctioned entity")
    
    # HIGH RISK FACTORS
    if mixer_hit:
        score += 40
        factors.append("🔄 Mixer/privacy service interaction")
    
    if any("unlimited approval" in f.lower() for f in flags):
        score += 30
        factors.append("⚠️ Unlimited token approvals")
    
    if any("circular" in f.lower() for f in flags):
        score += 25
        factors.append("⚠️ Circular flow pattern")
    
    # MEDIUM RISK FACTORS
    if complexity > 50:
        score += 20
        factors.append(f"📊 High complexity ({complexity}/100)")
    
    # Large value transfer check
    if any("large" in f.lower() or "high value" in f.lower() for f in flags):
        score += 15
        factors.append("💰 Large value transfer")
    
    # Timing risks
    if len(timing_flags) > 0:
        score += 10
        factors.append("🕐 Suspicious timing")
    
    # POSITIVE FACTORS (Mitigating factors)
    # Reduce risk if interacting with known safe exchanges
    if any("exchange" in label.lower() for label in entity_labels):
        score = max(0, score - 15)
        factors.append("✅ Known exchange interaction")
    
    return min(100, score), factors

def track_address(addresses_dict: Dict, address: str, label: str, tx_type: str):
    """Track unique addresses for graph generation"""
    if address not in addresses_dict:
        addresses_dict[address] = {
            'label': label,
            'type': tx_type,
            'count': 1
        }
    else:
        addresses_dict[address]['count'] += 1

def get_transaction_display_value(tx: Dict) -> str:
    """Get the display value for a transaction"""
    if tx.get('native_transfers'):
        for transfer in tx['native_transfers']:
            value = transfer.get('value_formatted', '0')
            if float(value) > 0:
                return f"{value} ETH"

    if tx.get('erc20_transfers'):
        for transfer in tx['erc20_transfers']:
            value = transfer.get('value_formatted', '0')
            symbol = transfer.get('token_symbol', 'TOKEN')
            if value and value != '0':
                return f"{value} {symbol}"

    eth_value = float(tx.get('value', 0)) / 1e18
    if eth_value > 0:
        return f"{eth_value:.6f} ETH"

    return tx.get('summary', '0 ETH')

def get_swap_details(tx: Dict) -> tuple[str, str]:
    """Extract swap from and to values"""
    swap_from = "Unknown"
    swap_to = "Unknown"

    if tx.get('erc20_transfers'):
        for transfer in tx['erc20_transfers']:
            value = transfer.get('value_formatted', '0')
            symbol = transfer.get('token_symbol', '')
            direction = transfer.get('direction', '')

            if direction == 'send' and value:
                swap_from = f"{value} {symbol}"
            elif direction == 'receive' and value:
                swap_to = f"{value} {symbol}"

    if swap_to == "Unknown" and tx.get('native_transfers'):
        for transfer in tx['native_transfers']:
            if transfer.get('direction') == 'receive':
                value = transfer.get('value_formatted', '0')
                swap_to = f"{value} ETH"

    return swap_from, swap_to

def get_nft_display_info(tx: Dict) -> str:
    """Get NFT display information"""
    if tx.get('nft_transfers'):
        nft = tx['nft_transfers'][0]
        token_id = nft.get('token_id', 'Unknown')
        return f"NFT #{token_id}"
    return "NFT"

def get_link_value(tx: Dict) -> str:
    """Get numerical value for link"""
    eth_value = float(tx.get('value', 0)) / 1e18
    return str(eth_value) if eth_value > 0 else "0"

def get_sublabel_for_type(node_type: str) -> str:
    """Get sublabel based on node type"""
    labels = {
        'incoming': '+',
        'outgoing': '-',
        'exchange': 'Swap',
        'contract': 'NFT'
    }
    return labels.get(node_type, '')

def get_color_for_type(node_type: str) -> str:
    """Get color for node type"""
    colors = {
        'main': '#3498db',
        'incoming': '#2ecc71',
        'outgoing': '#e74c3c',
        'exchange': '#f39c12',
        'contract': '#9b59b6'
    }
    return colors.get(node_type, '#95a5a6')

def shorten_address(address: str) -> str:
    """Shorten Ethereum address"""
    if not address:
        return "Unknown"
    if len(address) > 12:
        return f"{address[:6]}...{address[-4:]}"
    return address

def format_date(timestamp: str) -> str:
    """Format timestamp to readable date"""
    try:
        dt = datetime.fromisoformat(timestamp.replace('Z', '+00:00'))
        return dt.strftime('%b %d')
    except:
        return timestamp[:10]

def generate_graph_data(transactions: List[Dict], unique_addresses: Dict, main_address: str) -> tuple[List[GraphNode], List[GraphLink]]:
    """Generate nodes and links for network graph"""
    nodes = []
    links = []

    main_label = transactions[0].get('from_address_label', 'Main Wallet') if transactions else 'Main Wallet'
    nodes.append(GraphNode(
        id='main',
        label=main_label,
        sublabel=shorten_address(main_address),
        type='main',
        size=35,
        color='#3498db',
        glow=True
    ))

    sorted_addresses = sorted(
        unique_addresses.items(),
        key=lambda x: x[1]['count'],
        reverse=True
    )[:15]

    address_to_node_id = {}

    for idx, (addr, info) in enumerate(sorted_addresses):
        node_id = f"node_{idx}"
        address_to_node_id[addr] = node_id

        nodes.append(GraphNode(
            id=node_id,
            label=info['label'],
            sublabel=get_sublabel_for_type(info['type']),
            type=info['type'],
            size=22 + min(info['count'] * 2, 8),
            color=get_color_for_type(info['type'])
        ))

    for tx in transactions:
        category = tx.get('category', '')
        from_addr = tx['from_address']
        to_addr = tx['to_address']

        if category in ['receive', 'airdrop', 'token receive']:
            source_id = address_to_node_id.get(from_addr)
            target_id = 'main'
            link_type = 'incoming'
        elif category in ['send', 'token send']:
            source_id = 'main'
            target_id = address_to_node_id.get(to_addr)
            link_type = 'outgoing'
        elif category == 'token swap':
            source_id = 'main'
            target_id = address_to_node_id.get(to_addr)
            link_type = 'exchange'
        elif category in ['nft purchase', 'nft sale']:
            source_id = 'main'
            target_id = address_to_node_id.get(to_addr)
            link_type = 'contract'
        else:
            continue

        if source_id and target_id:
            links.append(GraphLink(
                source=source_id,
                target=target_id,
                value=get_link_value(tx),
                label=get_transaction_display_value(tx),
                type=link_type
            ))

    return nodes, links

def process_transactions(transactions: List[Dict], main_address: str) -> Dict:
    """Process and categorize transactions for the dashboard"""

    incoming = []
    outgoing = []
    swaps = []
    nfts = []

    total_in = 0.0
    total_out = 0.0
    swap_count = 0
    nft_count = 0

    unique_addresses = {}

    for tx in transactions:
        category = tx.get('category', 'unknown')
        tx_hash = tx['hash']

        if category in ['receive', 'airdrop']:
            from_label = tx.get('from_address_label') or shorten_address(tx['from_address'])
            amount = get_transaction_display_value(tx)

            incoming.append(DashboardTransaction(
                id=tx_hash[:10],
                hash=tx_hash,
                block_timestamp=tx['block_timestamp'],
                from_address=tx['from_address'],
                to_address=tx.get('to_address', ''),
                value=amount,
                date=format_date(tx['block_timestamp']),
                category=category,
                risk_score=0,
                flags=[],
                entity_interaction=from_label
            ))

            eth_value = float(tx.get('value', 0)) / 1e18
            if eth_value > 0:
                total_in += eth_value

            track_address(unique_addresses, tx['from_address'], from_label, 'incoming')

        elif category in ['send']:
            to_label = tx.get('to_address_label') or shorten_address(tx['to_address'])
            amount = get_transaction_display_value(tx)

            outgoing.append(DashboardTransaction(
                id=tx_hash[:10],
                hash=tx_hash,
                block_timestamp=tx['block_timestamp'],
                from_address=tx['from_address'],
                to_address=tx.get('to_address', ''),
                value=amount,
                date=format_date(tx['block_timestamp']),
                category=category,
                risk_score=0,
                flags=[],
                entity_interaction=to_label
            ))

            eth_value = float(tx.get('value', 0)) / 1e18
            if eth_value > 0:
                total_out += eth_value

            track_address(unique_addresses, tx['to_address'], to_label, 'outgoing')

        elif category == 'token swap':
            swap_from, swap_to = get_swap_details(tx)
            via = tx.get('to_address_label', '1inch')

            swaps.append(DashboardTransaction(
                id=tx_hash[:10],
                hash=tx_hash,
                block_timestamp=tx['block_timestamp'],
                from_address=tx['from_address'],
                to_address=tx.get('to_address', ''),
                value=f"{swap_from} → {swap_to}",
                date=format_date(tx['block_timestamp']),
                category=category,
                risk_score=0,
                flags=[],
                entity_interaction=via
            ))
            swap_count += 1

            track_address(unique_addresses, tx['to_address'], via, 'exchange')

        elif category in ['nft purchase', 'nft sale']:
            nft_info = get_nft_display_info(tx)
            action = 'Bought' if category == 'nft purchase' else 'Sold'

            nfts.append(DashboardTransaction(
                id=tx_hash[:10],
                hash=tx_hash,
                block_timestamp=tx['block_timestamp'],
                from_address=tx['from_address'],
                to_address=tx.get('to_address', ''),
                value=get_transaction_display_value(tx),
                date=format_date(tx['block_timestamp']),
                category=category,
                risk_score=0,
                flags=[action],
                entity_interaction=nft_info
            ))
            nft_count += 1

            to_label = tx.get('to_address_label', 'NFT Contract')
            track_address(unique_addresses, tx['to_address'], to_label, 'contract')

        elif category == 'token receive':
            from_label = tx.get('from_address_label') or shorten_address(tx['from_address'])
            amount = get_transaction_display_value(tx)

            incoming.append(DashboardTransaction(
                id=tx_hash[:10],
                hash=tx_hash,
                block_timestamp=tx['block_timestamp'],
                from_address=tx['from_address'],
                to_address=tx.get('to_address', ''),
                value=amount,
                date=format_date(tx['block_timestamp']),
                category=category,
                risk_score=0,
                flags=[],
                entity_interaction=from_label
            ))

            track_address(unique_addresses, tx['from_address'], from_label, 'incoming')

    graph_nodes, graph_links = generate_graph_data(transactions, unique_addresses, main_address)

    return {
        'stats': DashboardStats(
            total_in=f"{total_in:.4f} ETH",
            total_out=f"{total_out:.4f} ETH",
            swaps=swap_count,
            nfts=nft_count,
            incoming_count=len(incoming),
            outgoing_count=len(outgoing)
        ),
        'transactions': {
            'incoming': incoming[:7],
            'outgoing': outgoing[:4],
            'swaps': swaps[:5],
            'nfts': nfts[:5]
        },
        'graph_data': GraphData(
            nodes=graph_nodes,
            links=graph_links
        )
    }

# API Endpoints

@app.get("/api/analyze-transaction/{tx_hash}", response_model=AnalysisResult)
async def analyze_transaction(tx_hash: str, chain: str = "eth"):
    """
    Enhanced transaction analysis with deep event parsing and pattern detection
    
    - **tx_hash**: Transaction hash
    - **chain**: Blockchain network (default: eth)
    
    Returns comprehensive risk analysis including:
    - Event-level analysis (approvals, swaps, transfers)
    - Token flow graphs with hop tracking
    - Entity recognition via Moralis labels
    - Timing and complexity analysis
    - Multi-factor risk scoring
    """
    try:
        tx_data = moralis_request(
            f"/transaction/{tx_hash}/verbose",
            params={"chain": chain}
        )
        
        from_addr = tx_data.get("from_address", "")
        to_addr = tx_data.get("to_address", "")
        value_wei = int(tx_data.get("value", 0))
        value_eth = value_wei / 1e18 # Convert Wei to ETH
        nonce = int(tx_data.get("nonce", 0))
        
        # Initialize flags and checks
        flags = []
        
        # Critical Sanctions Check
        from_sanctioned, from_reason = check_sanctions(from_addr)
        to_sanctioned, to_reason = check_sanctions(to_addr)
        sanctions_hit = from_sanctioned or to_sanctioned
        
        if from_sanctioned:
            flags.append(f"🚨 CRITICAL: From address sanctioned - {from_reason}")
        if to_sanctioned:
            flags.append(f"🚨 CRITICAL: To address sanctioned - {to_reason}")
        
        # Extract Entity Labels (Mixers, Exchanges)
        entity_labels, mixer_hit, exchange_hit = extract_moralis_labels(tx_data)
        
        if mixer_hit:
            flags.append("🔄 Mixer/privacy service detected")
        
        # Deep Event Analysis (Logs)
        event_analyses, event_flags = analyze_events(tx_data.get("logs", []))
        flags.extend(event_flags)
        
        # Build Token Flow Graph
        token_flows, total_hops, flow_flags = build_token_flow_graph(
            tx_data.get("logs", [])
        )
        flags.extend(flow_flags)
        
        # Value Analysis
        if value_eth > 100:
            flags.append(f"💰 Very high value: {value_eth:.2f} ETH (~${value_eth * 2500:.2f})")
        elif value_eth > 10:
            flags.append(f"💰 High value: {value_eth:.2f} ETH")
        
        # Analyze Decoded Function Calls (Input Data)
        decoded_call = tx_data.get("decoded_call")
        if decoded_call:
            method = decoded_call.get("label", "").lower()
            
            # Check for high-risk methods
            if any(risk in method for risk in HIGH_RISK_METHODS):
                flags.append(f"🚨 HIGH RISK method: {decoded_call.get('label')}")
            
            # Check parameters specifically for 'deadline' (MEV detection)
            params = decoded_call.get("params", [])
            for param in params:
                if param.get("name") == "deadline":
                    # Very short deadline relative to timestamp might indicate MEV/Flashbots
                    deadline = int(param.get("value", 0))
                    if 0 < deadline < 9999999999: # Simple heuristic
                        flags.append("⚡ Short deadline (possible MEV)")
        
        # Timing Analysis
        timing_flags = analyze_timing(tx_data.get("block_timestamp", ""))
        
        # Calculate Complexity Score
        complexity_score = calculate_complexity_score(
            token_flows, 
            len(tx_data.get("logs", []))
        )
        
        # Calculate Overall Risk Score
        risk_score, risk_factors = calculate_advanced_risk_score(
            sanctions_hit,
            mixer_hit,
            flags,
            entity_labels,
            complexity_score,
            timing_flags
        )
        
        # Determine Categorical Risk Level
        if risk_score >= 70:
            risk_level = "CRITICAL"
        elif risk_score >= 50:
            risk_level = "HIGH"
        elif risk_score >= 30:
            risk_level = "MEDIUM"
        else:
            risk_level = "LOW"
        
        # Add 'Safe' indicator if score is low and no flags
        if not flags and risk_score < 30:
            flags.append("✅ Standard transaction - no suspicious indicators")
        
        # Construct and return result object
        return AnalysisResult(
            tx_hash=tx_hash,
            risk_score=risk_score,
            risk_level=risk_level,
            risk_factors=risk_factors,
            flags=flags,
            details=TransactionDetails(
                from_address=from_addr,
                from_label=tx_data.get("from_address_label"),
                from_entity=tx_data.get("from_address_entity"),
                to_address=to_addr,
                to_label=tx_data.get("to_address_label"),
                to_entity=tx_data.get("to_address_entity"),
                value=f"{value_eth:.6f} ETH",
                block_number=tx_data.get("block_number", 0),
                block_timestamp=tx_data.get("block_timestamp", ""),
                gas_used=tx_data.get("receipt_gas_used", "0"),
                gas_price=f"{int(tx_data.get('gas_price', 0)) / 1e9:.2f} Gwei", # Convert Wei to Gwei
                transaction_fee=tx_data.get("transaction_fee", "0"),
                nonce=nonce,
                decoded_call=decoded_call,
                token_flows=token_flows,
                total_hops=total_hops
            ),
            event_analysis=event_analyses,
            sanctions_check=sanctions_hit,
            mixer_interaction=mixer_hit,
            exchange_interaction=exchange_hit,
            entity_labels=entity_labels,
            complexity_score=complexity_score,
            timing_flags=timing_flags
        )
        
    except Exception as e:
        raise HTTPException(status_code=400, detail=f"Error analyzing transaction: {str(e)}")

@app.get("/api/analyze-address/{address}", response_model=AddressAnalysis)
async def analyze_address(address: str, chain: str = "eth", limit: int = 25):
    """
    Enhanced address analysis with behavioral pattern detection
    
    - **address**: Wallet address
    - **chain**: Blockchain network (default: eth)
    - **limit**: Number of transactions to analyze (default: 25)
    
    Returns comprehensive address profile including:
    - Transaction velocity and timing patterns
    - High-risk counterparty detection
    - Entity label extraction
    - Behavioral anomaly detection
    - Multi-factor risk scoring
    """
    try:
        tx_data = moralis_request(
            f"/{address}/verbose",
            params={"chain": chain, "limit": limit, "order": "DESC"}
        )
        
        transactions = tx_data.get("result", [])
        
        if not transactions:
            raise HTTPException(status_code=404, detail="No transactions found")
        
        # Initialize analysis counters and lists
        flags = []
        risk_factors = []
        high_risk_counterparties = []
        mixer_interactions = 0
        large_tx_count = 0
        total_volume = 0
        timestamps = []
        
        # Check if target address itself is sanctioned
        addr_sanctioned, addr_reason = check_sanctions(address)
        sanctions_hit = addr_sanctioned
        
        if addr_sanctioned:
            flags.append(f"🚨 CRITICAL: Address is sanctioned - {addr_reason}")
            risk_factors.append("Address on sanctions list")
        
        # Determine Label for the Target Address (from own txs)
        address_label = None
        if transactions:
            first_tx = transactions[0]
            # Check if address was sender in first tx
            if first_tx.get("from_address", "").lower() == address.lower():
                address_label = first_tx.get("from_address_label")
            else:
                address_label = first_tx.get("to_address_label")
        
        # Iterate and Analyze Recent Transactions
        recent_txs = []
        entity_interactions = defaultdict(int)
        
        for tx in transactions:
            tx_hash = tx.get("hash", "")
            from_addr = tx.get("from_address", "")
            to_addr = tx.get("to_address", "")
            value = int(tx.get("value", 0)) / 1e18 # ETH value
            timestamp = tx.get("block_timestamp", "")
            
            # Parse timestamp for timing analysis
            try:
                dt = datetime.fromisoformat(timestamp.replace("Z", "+00:00"))
                timestamps.append(dt)
            except:
                pass
            
            total_volume += value
            
            # Identify Counterparty (the other side of the tx)
            is_outgoing = from_addr.lower() == address.lower()
            counterparty = to_addr if is_outgoing else from_addr
            
            # Get Counterparty Labels
            if is_outgoing:
                cp_label = tx.get("to_address_label")
                cp_entity = tx.get("to_address_entity")
            else:
                cp_label = tx.get("from_address_label")
                cp_entity = tx.get("from_address_entity")
            
            # Track counts of potential entities interacted with
            if cp_entity:
                entity_interactions[cp_entity] += 1
            
            # Analyze this specific transaction
            tx_flags = []
            tx_risk = 0
            entity_info = None
            
            # Counterparty Sanctions Check
            cp_sanctioned, cp_reason = check_sanctions(counterparty)
            if cp_sanctioned:
                tx_flags.append(f"Sanctioned: {cp_reason}")
                high_risk_counterparties.append(counterparty)
                tx_risk += 70
            
            # Counterparty Mixer Check
            # Check label keywords
            if cp_label and any(kw in cp_label.lower() for kw in MIXER_KEYWORDS):
                tx_flags.append("Mixer interaction")
                mixer_interactions += 1
                tx_risk += 40
                entity_info = cp_label
            # Check entity keywords
            elif cp_entity and any(kw in cp_entity.lower() for kw in MIXER_KEYWORDS):
                tx_flags.append("Mixer interaction")
                mixer_interactions += 1
                tx_risk += 40
                entity_info = cp_entity
            
            # Transaction Value Check
            if value > 50:
                tx_flags.append(f"Very large: {value:.2f} ETH")
                large_tx_count += 1
                tx_risk += 25
            elif value > 10:
                tx_flags.append(f"Large: {value:.2f} ETH")
                large_tx_count += 1
                tx_risk += 15
            
            # Set display info for entity
            if cp_entity and not entity_info:
                entity_info = cp_entity
            elif cp_label and not entity_info:
                entity_info = cp_label
            
            if not tx_flags:
                tx_flags.append("Standard")
            
            # Add to list of analyzed transactions
            recent_txs.append(AddressTransaction(
                hash=tx_hash,
                block_timestamp=timestamp,
                from_address=from_addr,
                to_address=to_addr,
                value=f"{value:.4f} ETH",
                risk_score=min(100, tx_risk),
                flags=tx_flags,
                entity_interaction=entity_info
            ))
        
        # Time-Based Pattern Analysis (Bursts, Late Night)
        time_patterns = analyze_time_patterns(timestamps)
        
        # Generate Address-Level Aggregate Flags
        if mixer_interactions > 0:
            flags.append(f"🔄 Mixer interactions: {mixer_interactions} transaction(s)")
            risk_factors.append("Multiple mixer interactions")
        
        if large_tx_count > 3:
            flags.append(f"💰 Multiple large transactions: {large_tx_count} txs > 10 ETH")
            risk_factors.append("High-value transaction pattern")
        
        if time_patterns.burst_detected:
            flags.append(f"⚡ Burst activity detected: {time_patterns.time_details}")
            risk_factors.append("Burst transaction pattern")
        
        if time_patterns.suspicious_timing:
            flags.append("🌙 Unusual timing patterns detected")
            risk_factors.append("Off-hours activity")
        
        if high_risk_counterparties:
            unique_risk = len(set(high_risk_counterparties))
            flags.append(f"🚨 High-risk counterparties: {unique_risk} address(es)")
            risk_factors.append("Sanctioned counterparties")
        
        if total_volume > 500:
            flags.append(f"📊 Very high volume: {total_volume:.2f} ETH")
            risk_factors.append("Extremely high transaction volume")
        elif total_volume > 100:
            flags.append(f"📊 High volume: {total_volume:.2f} ETH")
        
        # Entity Interaction Summary
        entity_labels = []
        if address_label:
            entity_labels.append(f"🏷️ Address: {address_label}")
        
        # Get top 5 entities interacted with
        for entity, count in sorted(entity_interactions.items(), key=lambda x: x[1], reverse=True)[:5]:
            entity_labels.append(f"🔗 {entity} ({count} txs)")
        
        # Behavioral Summary Dict
        behavior_summary = {
            "total_volume_eth": round(total_volume, 4),
            "avg_tx_value_eth": round(total_volume / len(transactions), 4) if transactions else 0,
            "large_tx_count": large_tx_count,
            "mixer_interaction_count": mixer_interactions,
            "unique_counterparties": len(set(
                [tx.to_address for tx in recent_txs] + [tx.from_address for tx in recent_txs]
            )),
            "top_entities": dict(list(entity_interactions.items())[:5]),
            "analysis_period_days": (timestamps[0] - timestamps[-1]).days if len(timestamps) > 1 else 0
        }
        
        # Calculate Final Risk Score
        risk_score, _ = calculate_advanced_risk_score(
            sanctions_hit,
            mixer_interactions > 0,
            flags,
            entity_labels,
            min(len(transactions) * 2, 50),  # Use tx frequency as proxy for complexity
            [f for f in flags if "timing" in f.lower() or "late" in f.lower()]
        )
        
        # Small penalty for burst behavior
        if time_patterns.burst_detected:
            risk_score = min(100, risk_score + 15)
        
        # Determine Risk Level
        if risk_score >= 70:
            risk_level = "CRITICAL"
        elif risk_score >= 50:
            risk_level = "HIGH"
        elif risk_score >= 30:
            risk_level = "MEDIUM"
        else:
            risk_level = "LOW"
        
        if not flags:
            flags.append("✅ No suspicious patterns detected")
        
        # Return complete analysis
        return AddressAnalysis(
            address=address,
            address_label=address_label,
            total_transactions=len(transactions),
            risk_score=risk_score,
            risk_level=risk_level,
            risk_factors=risk_factors,
            flags=flags,
            entity_labels=entity_labels,
            recent_transactions=recent_txs,
            high_risk_counterparties=list(set(high_risk_counterparties)),
            sanctions_check=sanctions_hit,
            mixer_interaction=mixer_interactions > 0,
            time_patterns=time_patterns,
            behavior_summary=behavior_summary
        )
        
    except HTTPException:
        raise
    except Exception as e:
        raise HTTPException(status_code=400, detail=f"Error analyzing address: {str(e)}")

def analyze_time_patterns(timestamps: List[datetime]) -> TimePattern:
    """Analyze temporal patterns in transaction history (e.g., density, timing)"""
    if len(timestamps) < 2:
        return TimePattern(
            tx_per_hour=0,
            burst_detected=False,
            suspicious_timing=False,
            time_details="Insufficient data"
        )
    
    # Sort timestamps newest first
    timestamps = sorted(timestamps, reverse=True)
    
    # Calculate Velocity (TX/Hour)
    time_span_hours = (timestamps[0] - timestamps[-1]).total_seconds() / 3600
    tx_per_hour = len(timestamps) / time_span_hours if time_span_hours > 0 else 0
    
    # Detect Bursts: Check if 3 transactions happened within a 1-hour window
    burst_detected = False
    for i in range(len(timestamps) - 2):
        window = (timestamps[i] - timestamps[i+2]).total_seconds() / 3600
        if window < 1:
            burst_detected = True
            break
    
    # Check for Suspicious Timing (Late Night): >30% of txs between 2am and 5am
    suspicious_timing = sum(1 for ts in timestamps if 2 <= ts.hour <= 5) > len(timestamps) * 0.3
    
    # Text summary details
    details = f"{tx_per_hour:.2f} tx/hour over {time_span_hours:.1f} hours"
    if burst_detected:
        details += " | Burst detected"
    
    return TimePattern(
        tx_per_hour=round(tx_per_hour, 2),
        burst_detected=burst_detected,
        suspicious_timing=suspicious_timing,
        time_details=details
    )

@app.get("/api/wallet-history/{address}")
async def get_wallet_history(address: str, chain: str = "eth", limit: int = 25):
    """
    Get formatted wallet transaction history for dashboard visualization
    
    - **address**: Wallet address or ENS name
    - **chain**: Blockchain network (default: eth)
    - **limit**: Number of transactions to fetch (default: 25)
    
    Returns structured data optimized for frontend dashboard:
    - Categorized transactions (incoming, outgoing, swaps, NFTs)
    - Summary statistics
    - Network graph nodes and links
    """
    try:
        # Fetch wallet history from Moralis
        tx_data = moralis_request(
            f"/wallets/{address}/history",
            params={"chain": chain, "limit": limit, "order": "DESC"}
        )
        
        transactions = tx_data.get("result", [])
        
        if not transactions:
            raise HTTPException(status_code=404, detail="No transactions found for this address")
        
        # Initialize categorized lists
        incoming = []
        outgoing = []
        swaps = []
        nfts = []
        
        # Track nodes and links for network graph
        nodes_map = {}
        links_list = []
        
        # Normalize the target address for comparison
        target_address = address.lower()
        
        # Get address label if available
        main_label = None
        if transactions:
            first_tx = transactions[0]
            if first_tx.get("from_address", "").lower() == target_address:
                main_label = first_tx.get("from_address_label") or address
            else:
                main_label = first_tx.get("to_address_label") or address
        
        # Add main wallet node
        nodes_map["main"] = {
            "id": "main",
            "label": main_label or "Main Wallet",
            "sublabel": f"{address[:6]}...{address[-4:]}",
            "type": "main",
            "address": address
        }
        
        # Process each transaction
        for tx in transactions:
            tx_hash = tx.get("hash", "")
            from_addr = tx.get("from_address", "")
            to_addr = tx.get("to_address", "")
            value = float(tx.get("value", 0)) / 1e18
            timestamp = tx.get("block_timestamp", "")
            category = tx.get("category", "")
            summary = tx.get("summary", "")
            
            # Format date
            try:
                dt = datetime.fromisoformat(timestamp.replace("Z", "+00:00"))
                date_str = dt.strftime("%b %d")
            except:
                date_str = "Unknown"
            
            # Determine if transaction is incoming or outgoing
            is_incoming = to_addr.lower() == target_address
            is_outgoing = from_addr.lower() == target_address
            
            # Get counterparty info
            counterparty_addr = from_addr if is_incoming else to_addr
            counterparty_label = None
            counterparty_entity = None
            
            if is_incoming:
                counterparty_label = tx.get("from_address_label")
                counterparty_entity = tx.get("from_address_entity")
            else:
                counterparty_label = tx.get("to_address_label")
                counterparty_entity = tx.get("to_address_entity")
            
            # Use entity or label, fallback to shortened address
            display_name = counterparty_entity or counterparty_label or f"{counterparty_addr[:6]}...{counterparty_addr[-4:]}"
            
            # Process NFT transactions
            nft_transfers = tx.get("nft_transfers", [])
            if nft_transfers and len(nft_transfers) > 0:
                for nft in nft_transfers:
                    token_id = nft.get("token_id", "Unknown")
                    direction = nft.get("direction", "").lower()
                    collection_name = nft.get("token_name", "NFT")
                    
                    if direction == "receive":
                        action = "Bought"
                    elif direction == "send":
                        action = "Sold"
                    else:
                        action = "Traded"
                    
                    nfts.append({
                        "id": tx_hash,
                        "action": action,
                        "item": f"{collection_name} #{token_id}",
                        "amount": f"{value:.5f} ETH" if value > 0 else "Free",
                        "date": date_str,
                        "hash": tx_hash
                    })
                    
                    # Add NFT node for graph
                    nft_node_id = f"nft_{token_id}_{tx_hash[:8]}"
                    if nft_node_id not in nodes_map:
                        nodes_map[nft_node_id] = {
                            "id": nft_node_id,
                            "label": f"{collection_name}",
                            "sublabel": f"#{token_id}",
                            "type": "contract",
                            "address": nft.get("token_address", "")
                        }
                    
                    # Add link
                    if action == "Bought":
                        links_list.append({
                            "source": "main",
                            "target": nft_node_id,
                            "value": str(value),
                            "label": f"{value:.3f} ETH",
                            "type": "outgoing"
                        })
                    else:
                        links_list.append({
                            "source": nft_node_id,
                            "target": "main",
                            "value": str(value),
                            "label": f"{value:.3f} ETH",
                            "type": "incoming"
                        })
            
            # Process token swaps
            erc20_transfers = tx.get("erc20_transfers", [])
            if category == "token swap" or (len(erc20_transfers) > 1):
                # It's a swap
                sent_tokens = [t for t in erc20_transfers if t.get("direction") == "send"]
                received_tokens = [t for t in erc20_transfers if t.get("direction") == "receive"]
                
                if sent_tokens and received_tokens:
                    from_token = sent_tokens[0]
                    to_token = received_tokens[0]
                    
                    from_amount = from_token.get("value_formatted", "0")
                    from_symbol = from_token.get("token_symbol", "TOKEN")
                    to_amount = to_token.get("value_formatted", "0")
                    to_symbol = to_token.get("token_symbol", "TOKEN")
                    
                    via_label = counterparty_label or "DEX"
                    
                    swaps.append({
                        "id": tx_hash,
                        "from": f"{from_amount} {from_symbol}",
                        "to": f"{to_amount} {to_symbol}",
                        "via": via_label,
                        "date": date_str,
                        "hash": tx_hash
                    })
                    
                    # Add swap node for graph
                    swap_node_id = f"swap_{counterparty_addr[:10]}"
                    if swap_node_id not in nodes_map:
                        nodes_map[swap_node_id] = {
                            "id": swap_node_id,
                            "label": via_label,
                            "sublabel": "Swap",
                            "type": "exchange",
                            "address": counterparty_addr
                        }
                        
                        links_list.append({
                            "source": "main",
                            "target": swap_node_id,
                            "value": from_amount,
                            "label": "Swap",
                            "type": "exchange"
                        })
            
            # Process regular incoming transactions
            elif is_incoming and category != "token swap":
                # Check for token transfers
                if erc20_transfers:
                    token = erc20_transfers[0]
                    amount_str = f"{token.get('value_formatted', '0')} {token.get('token_symbol', 'TOKEN')}"
                else:
                    amount_str = f"{value:.5f} ETH"
                
                incoming.append({
                    "id": tx_hash,
                    "from": display_name,
                    "amount": amount_str,
                    "date": date_str,
                    "hash": tx_hash
                })
                
                # Add node for graph
                node_id = f"in_{counterparty_addr[:10]}"
                if node_id not in nodes_map:
                    nodes_map[node_id] = {
                        "id": node_id,
                        "label": display_name,
                        "sublabel": f"+{amount_str}",
                        "type": "incoming",
                        "address": counterparty_addr
                    }
                    
                    links_list.append({
                        "source": node_id,
                        "target": "main",
                        "value": str(value),
                        "label": amount_str,
                        "type": "incoming"
                    })
            
            # Process regular outgoing transactions
            elif is_outgoing and category != "token swap" and not nft_transfers:
                # Check for token transfers
                if erc20_transfers:
                    token = erc20_transfers[0]
                    amount_str = f"{token.get('value_formatted', '0')} {token.get('token_symbol', 'TOKEN')}"
                else:
                    amount_str = f"{value:.5f} ETH"
                
                outgoing.append({
                    "id": tx_hash,
                    "to": display_name,
                    "amount": amount_str,
                    "date": date_str,
                    "hash": tx_hash
                })
                
                # Add node for graph
                node_id = f"out_{counterparty_addr[:10]}"
                if node_id not in nodes_map:
                    nodes_map[node_id] = {
                        "id": node_id,
                        "label": display_name,
                        "sublabel": f"-{amount_str}",
                        "type": "outgoing",
                        "address": counterparty_addr
                    }
                    
                    links_list.append({
                        "source": "main",
                        "target": node_id,
                        "value": str(value),
                        "label": amount_str,
                        "type": "outgoing"
                    })
        
        # Calculate summary statistics
        total_incoming = len(incoming)
        total_outgoing = len(outgoing)
        total_swaps = len(swaps)
        total_nfts = len(nfts)
        
        # Calculate total ETH values
        incoming_eth = sum(float(tx.get("value", 0)) / 1e18 for tx in transactions 
                          if tx.get("to_address", "").lower() == target_address)
        outgoing_eth = sum(float(tx.get("value", 0)) / 1e18 for tx in transactions 
                          if tx.get("from_address", "").lower() == target_address)
        
        # Build response
        return {
            "wallet_address": address,
            "wallet_label": main_label,
            "total_transactions": len(transactions),
            "summary": {
                "incoming": {
                    "count": total_incoming,
                    "total_eth": round(incoming_eth, 4)
                },
                "outgoing": {
                    "count": total_outgoing,
                    "total_eth": round(outgoing_eth, 4)
                },
                "swaps": {
                    "count": total_swaps
                },
                "nfts": {
                    "count": total_nfts
                }
            },
            "transactions": {
                "incoming": incoming[:10],  # Limit to top 10 for display
                "outgoing": outgoing[:10],
                "swaps": swaps[:10],
                "nfts": nfts[:10]
            },
            "network_graph": {
                "nodes": list(nodes_map.values()),
                "links": links_list
            },
            "cursor": tx_data.get("cursor"),
            "has_more": bool(tx_data.get("cursor"))
        }
        
    except HTTPException:
        raise
    except Exception as e:
        raise HTTPException(status_code=400, detail=f"Error fetching wallet history: {str(e)}")

@app.get("/")
def root():
    """Root endpoint to verify service status and capabilities"""
    return {
        "service": "Blockchain Forensics & Compliance APP",
        "version": "1.0.0",
        "provider": "Moralis",
        "features": [
            "Deep event analysis (all event types)",
            "Token flow graph construction",
            "Multi-factor risk scoring",
            "Behavioral pattern detection",
            "Entity label extraction",
            "Temporal anomaly detection"
        ],
        "endpoints": {
            "analyze_transaction": "/api/analyze-transaction/{tx_hash}",
            "analyze_address": "/api/analyze-address/{address}",
            "health": "/health",
            "docs": "/docs"
        },
        "status": "operational"
    }

@app.get("/health")
def health():
    """Health check endpoint containing a live Moralis connectivity test"""
    try:
        # Test connection by fetching latest block
        moralis_request("/block/latest", params={"chain": "eth"})
        return {
            "status": "healthy",
            "moralis_connected": True,
            "api_version": "1.0.0",
            "timestamp": datetime.utcnow().isoformat()
        }
    except:
        return {
            "status": "degraded",
            "moralis_connected": False,
            "timestamp": datetime.utcnow().isoformat()
        }

if __name__ == "__main__":
    import uvicorn
    print("🚀 Starting Blockchain Forensics APP...")
    print("📊 Features: Deep event analysis, token flow graphs, behavioral patterns")
    print("🔗 Moralis integration enabled")
    print("📖 API docs: http://localhost:8002/docs")
    uvicorn.run(app, host="0.0.0.0", port=8002)